# Machine Learning Via Spark


In this notebook, we will explore how Spark can assist us in machine learning applications and provide us with additional tools to address ML challenges.

## Using the Machine Learning library (MLlib)

Spark provides a library with ML functionality. The set of tools is ever expanding - see the latest at https://spark.apache.org/docs/latest/ml-guide.html

The library is implemented in Scala, and has python binding (i.e. calling from python to the API).


Using MLFlow (https://mlflow.org/docs/latest/python_api/mlflow.spark.html?highlight=spark#module-mlflow.spark)is also possible, but not covered here.

**For more information and explanations, check chapters 24 & 25 in** : Chambers, B., & Zaharia, M. (2018). "Spark: The definitive guide: Big data processing made simple" [SDG].

SDG serves as a comprehensive guide to Apache Spark, a powerful open-source distributed computing system used for big data processing and analytics.

You can find this book here:
https://github.com/VolodymyrGavrysh/My_RoadMap_Data_Science/blob/master/kds/books/Spark-The%20Definitive%20Guide.pdf

## Prepare the data

If a specific tool is not part of MLlib, maybe someone already implemented it.

Always be suspicious of the code source: who wrote it? when was the last update? how many stars?

See for example https://spark-packages.org/?q=tags%3A%22Machine%20Learning%22 which is a repo without any quality assurance. You can find a great code, a buggy code, or malware.

We deal with a classic data-set example of retail-data, capturing transactions and sales activities. Each record represents a unique transaction within a retail environment during the year 2011.

In [ ]:
from pathlib import Path
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MLlib').getOrCreate()

In [ ]:
def load_data(file_name_glob):
    """ load the contents of the input files.
        If we already saved them in Parquet file, use it.
        #  >>> load_data('../data/sdg/retail-data/by-day/2010-12*.csv')
        :param file_name_glob wildcard value of the files to read. e.g. "/mnt/dir/data*"
        :return: DataFrame containing all the data
    """
    
    def cache_file_name(file_name):
        t = file_name.replace('*',"_").replace('?',"_")
        return t[: t.rfind('.')] + ".parquet"
    
    import os
    dirname = os.path.dirname(file_name_glob)
    p = Path(dirname)
    fname = Path(file_name_glob)
    basename = fname.name
    cache_name = cache_file_name(file_name_glob)
    if Path(cache_name).exists():
        return spark.read.parquet(cache_name)
    
    #suffix = fname.suffix
    if not p.exists():
        raise ValueError('Path not found')
    file_list = list(p.glob(basename))
    x = [ str(f.resolve()) for f in file_list]
    df = spark.read \
    .option("header","true")\
    .option("inferSchema", "true")\
    .csv(x)

    # df.write.parquet(cache_name) - for optimization
    return df
    

df = load_data('../data/sdg/retail-data/by-day/2011-*.csv')
print(f"df.count = {df.count()}")


In [ ]:
df.show()

In [ ]:
df.printSchema()

Add a new column: "day of week" and split to train/test

In [ ]:
from pyspark.sql.functions import date_format, col
preppedDataFrame = df\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))
  #.coalesce(5)

# split to train and test:
trainDataFrame,testDataFrame  = preppedDataFrame.randomSplit([0.7, 0.3])

# we could also split using other criteria:
# trainDataFrame = preppedDataFrame\
#   .where("InvoiceDate < '2011-07-01'")
# testDataFrame = preppedDataFrame\
#   .where("InvoiceDate >= '2011-07-01'")

print(f"train:test ratio: {trainDataFrame.count()/testDataFrame.count()}")

#### Transformations for the data
Convert "day of week" into one hot vector. E.g "Mon" -> 2 -> one hot encoding
Convert "Country" -> "country_index"
Convert "UnitPrice", "Quantity", "day_of_week_encoded" columns into a single feature vector -> "features"

**What is the downside of using one hot encoding?**
- wasted space -> solved by using sparse vectors
- increased dimension

In [ ]:
from pyspark.ml.feature import StringIndexer
day_indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_week_index")

country_indexer = StringIndexer()\
  .setInputCol("Country")\
  .setOutputCol("country_index")

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_week_encoded")
from pyspark.ml.feature import VectorAssembler

#  add "features" column that contains the input columns as elements in a vector.
# Not very exciting, right?
vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")

# Read about pipelines here: https://spark.apache.org/docs/latest/ml-pipeline.html
from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([day_indexer, country_indexer, encoder, vectorAssembler])

fittedPipeline = transformationPipeline.fit(trainDataFrame)
transformedTraining = fittedPipeline.transform(trainDataFrame)
tranformedTest = fittedPipeline.transform(testDataFrame)

# Let's drop unused columns. 
# This reduces the amount of needed memory so improving performance.
transformedTraining = transformedTraining.drop('day_of_week').drop('day_of_week_encoded').drop('day_of_week_index'). drop('CustomerID')

# Apply the fitted pipeline to the test data
transformedTest = fittedPipeline.transform(testDataFrame)

# Drop unused columns from transformedTest
transformedTest = transformedTest.drop('day_of_week').drop('day_of_week_encoded').drop('day_of_week_index').drop('CustomerID')

The column "features" in the data frames was created through feature engineering technique - vectorization using Spark's VectorAssembler.

In [ ]:
# Caching the transfored DF will save a lot of time when reusing it (e.g. for hyper param tuning)
transformedTraining.cache()

In [ ]:
transformedTraining.printSchema()

## Classification using Logistic Regression


**Reminder**: Logistic regression is a fundamental statistical method used for binary classification tasks.

Let's examine the performance of the logistic regression model by predicting the country indices, with only the features in the test set.

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Create a logistic regression model
lr = LogisticRegression(labelCol="country_index", featuresCol="features")

# Fit the logistic regression model to the training data
fittedLR = lr.fit(transformedTraining)

# Make predictions on the test data and select relevant columns
predictions = fittedLR.transform(tranformedTest).select("country_index", "prediction")

# Group predictions by country index and calculate average prediction
grouped_predictions = predictions.groupBy("country_index").avg("prediction")

# Show the results for the top 20 country indices
grouped_predictions.show(20)


We can evaluate the model's performance:

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="country_index", predictionCol="prediction", metricName="accuracy")

# Calculate accuracy
accuracy = evaluator.evaluate(predictions)

print("Accuracy:", accuracy)


## Clustering using K-means


**Reminder**: K-means is an unsupervised machine learning algorithm used for clustering data into distinct groups, by iteratively assigning data points to the nearest cluster centroid, and updating the centroids to minimize the within-cluster sum of squares.

In Spark, training machine learning models is a two phase process. First we initialize an untrained model, then we
train it. There are always two types for every algorithm in MLlib’s DataFrame API. The algorithm Kmeans and then the
trained version which is a KMeansModel.

The column "features" contains the transformed features that will be used by the KMeans algorithm for clustering.


In [ ]:
from pyspark.ml.clustering import KMeans

# Define the KMeans model with k=6 clusters and a seed value for reproducibility
kmeans = KMeans().setK(6).setSeed(1)

# Fit the KMeans model to the transformed training data
kmModel = kmeans.fit(transformedTraining)

# Make predictions on the transformed test data
predictions = kmModel.transform(transformedTest)

When we call the fit() method with transformedTraining, KMeans utilizes the "features" column to cluster the data points based on the features contained within it. The algorithm finds the optimal cluster centroids based on these features.

In [ ]:
# Shows the result.
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for i, center in enumerate(centers):
    print(f"{i+1}: {center}")

Let's evaluate the model's performance:

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

# Compute the Silhouette score
silhouette_score = evaluator.evaluate(predictions)
print("Silhouette Score:", silhouette_score)

### Repeating the Runs

In the context of machine learning models like K-means clustering, ensuring repeatability and reproducibility of results is crucial for reliability and trustworthiness of the model.

**Repeatability** refers to the ability to obtain the same results by running the model multiple times with the same initial conditions, including the random seed. It is important for ensuring consistency in model performance and behavior.

**Reproducibility**, on the other hand, is the ability to obtain similar results even when different random numbers are used. In other words, if the model is run with different random seeds, but under the same conditions, one should still expect to get comparable outcomes. Reproducibility is essential for confirming the robustness of the model and its generalizability across different random initializations.

In the context of K-means clustering, where initial centroids are initialized randomly, ensuring both repeatability and reproducibility is vital. While setting a random seed (e.g., using `setSeed()` function) helps achieve repeatability, validating the model's robustness against different initializations is equally important for assessing its reproducibility.

By repeating runs with varying seeds and observing consistency or variance in results, we can gain insights into the stability and reliability of the K-means clustering algorithm. This process not only validates the model but also helps identify potential biases or sensitivities to initial conditions, contributing to the overall trustworthiness of the clustering results.


In [ ]:
# Define a list of seed values for testing repeatability and reproducibility
seed_values = [1, 42, 123]

for seed in seed_values:
    print(f"Running with seed: {seed}")
    # Define the KMeans model with k=6 clusters and a seed value for reproducibility
    kmeans = KMeans().setK(6).setSeed(seed)

    # Fit the KMeans model to the transformed training data
    kmModel = kmeans.fit(transformedTraining)

    # Make predictions on the transformed test data
    predictions = kmModel.transform(transformedTest)

    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator()

    # Compute the Silhouette score
    silhouette_score = evaluator.evaluate(predictions)
    print("Silhouette Score:", silhouette_score)
    print("\n")


The consistent high Silhouette scores obtained across runs with different seeds provide evidence for both repeatability and reproducibility of the KMeans clustering model.

To explore further opportunities in classifications & regressions, please look at:
https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression

## Model Selection and Hyperparameter Tuning

**Reminder**: Hyperparameters are settings that govern the learning process and affect the model's behavior and performance.

Hyperparameter tuning is essential because it helps optimize machine learning models' performance by finding the most effective combination of hyperparameters.

Using grid (cartesian product) is inefficient due to a lot of wasted runs. There are more advanced methods to reduce the tested parameters combinations.

**ParamGridBuilder**: The ParamGridBuilder is used to define a grid of hyperparameters for the model. In this example, we define a grid for the maximum number of iterations (maxIter) and regularization parameter (regParam) for the logistic regression model.

**CrossValidator**: CrossValidator is used to perform model selection and hyperparameter tuning. It takes an estimator (in this case, logistic regression), a set of hyperparameters, an evaluator, and the number of folds for cross-validation.

Exploring a Grid Search Technique to Construct a Logistic Regression Model and Investigate Diverse Hyperparameter Configurations Methodically:

*Note*: Beware of resource shortage! the following code will run 4 models.

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator , MulticlassClassificationEvaluator

# Define the Logistic Regression model
lr = LogisticRegression(labelCol="country_index", featuresCol="features")

# Define the parameter grid to search
param_grid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [50, 150]) \
    .addGrid(lr.regParam, [0.05, 1.0]) \
    .build()

# Define the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="country_index", metricName="accuracy")

# Define the cross-validator
cross_validator = CrossValidator(estimator=lr,
                                 estimatorParamMaps=param_grid,
                                 evaluator=evaluator,
                                 numFolds=3)

# Fit the model using cross-validation
cv_model = cross_validator.fit(transformedTraining)

# Make predictions on the test data
predictions = cv_model.transform(tranformedTest)

# Evaluate the model
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)


In [ ]:
import pprint
pprint.pprint(cv_model.explainParams())

To explore further opportunities in tuning, please look at:
https://spark.apache.org/docs/latest/ml-tuning.html

## Dimensionality Reduction using PCA

**Reminder**: Principal Component Analysis (PCA) is a widely used dimensionality reduction technique that aims to capture the most important information in a dataset by transforming the original features into a new set of orthogonal variables called principal components.

Let's perform PCA on the "features" column in our training data.

In [ ]:
from pyspark.ml.feature import PCA

# Assuming you have assembled features into a vector column named "features"
# You may need to adjust the inputCol based on your actual data
assembled_features = transformedTraining.select("features")

# Define the PCA model with desired number of principal components
num_principal_components = 3  # Don't use 2 components automatically. We live in a 3D world.
pca = PCA(k=num_principal_components, inputCol="features", outputCol="pca_features")

# Fit the PCA model to the data
pca_model = pca.fit(assembled_features)

# Transform the data to the lower-dimensional space
transformed_data = pca_model.transform(assembled_features)

# Show the transformed data with PCA features
transformed_data.select("pca_features").show(10, truncate=False)


#### Plotting

In [ ]:
import matplotlib.pyplot as plt

# Assuming transformed_data is your DataFrame with PCA features
pca_features = transformed_data.select("pca_features").collect()

# Extracting PCA feature values
pca_values = [row.pca_features.toArray() for row in pca_features]

# Creating a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Extracting PCA components
x = [val[0] for val in pca_values]
y = [val[1] for val in pca_values]
z = [val[2] for val in pca_values]

# Plotting data points
ax.scatter(x, y, z)

# Setting labels
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

# Displaying the plot
plt.show()


To explore further opportunities in dimensionality reduction, please look at:
https://spark.apache.org/docs/latest/mllib-dimensionality-reduction.html


## Basic Statistics

Let's demonstrate how to perform correlation analysis.

**Reminder**: Correlation analysis is a statistical technique used to quantify the degree to which two or more variables are related to each other. It helps identify patterns, relationships, and dependencies between different variables in a dataset. Correlation analysis is essential for understanding the interdependencies between features and for feature selection in machine learning models.

In [ ]:
from pyspark.ml.stat import Correlation

# Compute the correlation matrix for the "features" column
corr_matrix = Correlation.corr(transformedTraining, "features").head()

# Extract the correlation matrix as a DenseMatrix
corr_matrix_values = corr_matrix[0].toArray()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a heatmap using Seaborn
plt.figure(figsize=(7, 7))
sns.heatmap(corr_matrix_values, annot=True, cmap="coolwarm", fmt=".2f", square=True,
            xticklabels=transformedTraining.columns, yticklabels=transformedTraining.columns)

plt.title("Correlation Heatmap")
plt.show()

To explore further opportunities in statistics, please look at:
 https://spark.apache.org/docs/latest/ml-statistics.html

In [ ]:
# Do not forget to release the resources held by Spark
spark.stop()